## 2019

In [1]:
import pandas as pd
import numpy as np

df2019 = pd.read_excel('./data/2019.xlsx', sheet_name= 'Sheet1', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=55)
df2019.columns = ['팀 코드', '주제명', '팀원1', '팀원2', '팀원3']

df2019.head()

,팀 코드,주제명,팀원1,팀원2,팀원3
0,가-1,어떻게 해수욕장 인구를 효율적으로 측정할까?,2115 박장현,2117 송지원,NaN
1,가-2,"해수욕장, 언제 갈래요?",2401 곽서현,2402 박지윤,NaN
2,가-3,이안류 위험도를 예측하여 대비할 수 있을까?,2403 이영진,2503 박소예,NaN
3,가-4,어떻게 낚시 이용자를 증대시키고 해얀지역 경제를 활성화시킬 수 있을까?,2506 진혜민,2510 노윤호,NaN
4,가-5,해수욕장을 100%로 즐기려면?,2208 김도훈,2314 이정홍,NaN


In [2]:
df2019['팀 코드'] = df2019['팀 코드'].str.replace('가', 'A')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('나', 'B')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('다', 'C')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('라', 'D')

df2019['팀 코드'] = df2019['팀 코드'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))

df2019['팀원1학번'], df2019['팀원1이름'] = df2019['팀원1'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원1'].apply(lambda x: x[4:] if x is not np.nan else x)

df2019['팀원2학번'], df2019['팀원2이름'] = df2019['팀원2'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원2'].apply(lambda x: x[4:] if x is not np.nan else x)

df2019['팀원3학번'], df2019['팀원3이름'] = df2019['팀원3'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원3'].apply(lambda x: x[4:] if x is not np.nan else x)
df2019.drop(['팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df2019.tail()

,팀 코드,주제명,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
48,D08,어선의 최적항로는 어떤 모양일까?,3408,김장현,3409,김재영,NaN,NaN
49,D09,소형 배로 적조 문제를 해결하고 해양데이터까지 수집하면 어떨까?,3416,서영국,3504,공명준,NaN,NaN
50,D10,뱃멀미를 어떻게 예방할 수 있을까?,1110,배시우,1216,이상민,1518,조성현
51,D11,해류와 암초의 영향을 고려한 선박의 최적 경로는?,1108,박정원,1109,박주영,1415,이유신
52,D12,최적의 선박 항로는? 골든 타임 확보를 위한 선박의 안전성 보장,1406,김원준,1419,최강훈,NaN,NaN


In [3]:
properties = ['팀 코드', '주제명', '팀원1 학번', '팀원1 이름', '팀원2 학번', '팀원2 이름', '팀원3 학번', '팀원3 이름']

def get_student_number(student_number):
  try:
    return str(int(student_number))
  except:
    return '0000'

def set_metadata(team_info, document, year = 2019):
  document.metadata = {}
  team_info.fillna('0')

  document.metadata['Year'] = str(year)
  document.metadata['Team code'] = str(team_info['팀 코드'])
  document.metadata['Title'] = str(team_info['주제명'])

  try:
    document.metadata['Team name'] = str(team_info['팀명'])
  except:
    document.metadata['Team name'] = '0'

  document.metadata['Teammate #1 name'] = str(team_info['팀원1이름'])
  document.metadata['Teammate #1 number'] = get_student_number(team_info['팀원1학번'])
  document.metadata['Teammate #2 name'] = str(team_info['팀원2이름'])
  document.metadata['Teammate #2 number'] = get_student_number(team_info['팀원2학번'])
  document.metadata['Teammate #3 name'] = str(team_info['팀원3이름'])
  document.metadata['Teammate #3 number'] = get_student_number(team_info['팀원3학번'])
  
  try:
    document.metadata['Youtube link'] = str(team_info['YOUTUBE 영상 주소'])
  except:
    document.metadata['Youtube link'] = '0'

  return document




In [4]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 학번 추출

PDF_DIR = 'OceanICT_Data/2019_OceanICT/포스터/'
files = os.listdir(PDF_DIR)
file_list = sorted(files, key = lambda x: int(p.match(x).group()))

split_documents19 = []

for f in file_list:
  loader = PyMuPDFLoader(PDF_DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df2019.loc[df2019[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  try:
    new_document = set_metadata(result.iloc[0], document)
    split_documents19.append(new_document)
  except:
    continue

In [5]:
print(len(split_documents19))

54


In [6]:
from pprint import pprint

type(split_documents19[0])

langchain_core.documents.base.Document

## 2020

In [7]:
df20 = pd.read_excel('./data/2020.xlsx', sheet_name= '팀 코드 정리', usecols=[i + 8 for i in range(7)], skiprows=3, nrows=72)
df20.drop(['유형'], axis=1, inplace=True)
df20 = df20.rename(columns={'연구 제목': '주제명'})
df20.tail()

,팀원1,팀원2,팀원3,주제명,팀 번호,YOUTUBE 영상 주소
66,2406 김근형,2409 김준우,NaN,효율적인 등대 배치를 위한 통계적인 최적화 방법에 대한 탐구,D-8,https://youtu.be/yXW9bSgih-Y
67,2407 김민성,2412 류운비,NaN,연안에서 최적의 선박 회피경로 구하기,D-9,https://youtu.be/7qKrCrHhFQc
68,2520 최호영,NaN,NaN,선박 간 정보공유 플랫폼 구축을 통한 어획의 편의성 증대,D-10,https://youtu.be/SB1BPdxQ-h4
69,3214 정상,3314 한경찬,NaN,선박에서 코로나바이러스 피해를 줄일 수 있을까?,D-11,https://youtu.be/sV8qrHp3vcs
70,3307 박관우,3407 박성현,NaN,선박 사고 시 가장 효율적인 대처 방안 모색,D-12,https://youtu.be/-tG4NkGq__4


In [8]:
df20['팀 코드'] = df20['팀 번호'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))

for i in range(1, 4):
    df20[f'팀원{i}학번'], df20[f'팀원{i}이름'] = df20[f'팀원{i}'].apply(lambda x: x[:4] if x is not np.nan else x), df20[f'팀원{i}'].apply(lambda x: x[5:] if x is not np.nan else x)

df20.drop(['팀 번호', '팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df20.head()

,주제명,YOUTUBE 영상 주소,팀 코드,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
0,해운대 해수욕장 파속측정을 통한 이안류 발생시 알림 수신 프로그래밍,https://www.youtube.com/watch?v=JSnm4dtmU5c,A01,1118,이현승,1115,이영진,1308,김정현
1,해양데이터를 이용해 이안류 피해를 줄일 수 있을까?,https://www.youtube.com/watch?v=Y5YvaB7awd4,A02,1210,안동준,1212,이건희,1215,장연수
2,해양 양식장 관리 시스템 설계,https://youtu.be/En4Bn1UEuyw,A03,1216,정민서,1418,장문정,1515,이승헌
3,수온에 따른 어획량예측,https://youtu.be/eNqs5N7W9v8,A04,1311,박정윤,1312,백재승,1320,정윤상
4,해수욕장 인명사고 예방을 위한 경보 시스템 제작,https://www.youtube.com/watch?v=m4bCSDfgRaA,A05,2103,안주연,2104,이수민,NaN,NaN


In [9]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import tqdm

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 팀코드 추출

DIR = 'OceanICT_Data/2020_OceanICT/'
file_list = os.listdir(DIR)
file_list_pdf = [file for file in file_list if file.endswith(".pdf")]
file_list_pptx = [file for file in file_list if file.endswith(".pptx")]


split_documents20 = []

for f in file_list_pdf:
  loader = PyMuPDFLoader(DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df20.loc[df20[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  new_document = set_metadata(result.iloc[0], document, 2020)
  split_documents20.append(new_document)

for f in tqdm.tqdm(file_list_pptx):
  loader = UnstructuredPowerPointLoader(DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df20.loc[df20[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  new_document = set_metadata(result.iloc[0], document, 2020)
  split_documents20.append(new_document)

100%|██████████| 70/70 [00:07<00:00,  9.75it/s]


In [10]:
len(split_documents20)

74

## 2021

In [11]:
df21_A = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 문화와 관광 진흥', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=20)
df21_B = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 생태계 및 환경 보존', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=30)
df21_C = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 자원의 이용 기반 구축', usecols=[i + 2 for i in range(5)], skiprows=2, nrows=34)
df21_D = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 선박 관련 기술', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=10)

df21 = pd.concat([df21_A, df21_B, df21_C, df21_D])
df21 = df21.rename(columns={'주제': '주제명'})
df21.head()

,연번,주제명,팀원1,팀원2,팀원3
0,A-1,해변 관광환경 개선 및 만족도 향상을 위한 해변 추천 서비스 제작하기,1413 신재현,1420 하승훈,NaN
1,A-2,해양 정보를 이용한 플레이리스트 추천 프로그램,2104 윤현서,2105 이다은,NaN
2,A-3,인공지능과 오픈 API를 이용한 해양 경보 어플리케이션 제작,2108 김경태,2119 최준영,NaN
3,A-4,부산 수산시장의 해산물 가격 판단 서비스,2213 안동준,2215 장연수,NaN
4,A-5,역기압 효과를 이용한 파고 높이 계산을 통해 어떻게 풍랑주의보 알리미 제작할 수 있을까?,2217 정재민,2218 조현준,NaN


In [12]:
df21['팀 코드'] = df21['연번'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))
df21['연번'] = df21['연번'].str.replace('-', '')

for i in range(1, 4):
    df21[f'팀원{i}학번'], df21[f'팀원{i}이름'] = df21[f'팀원{i}'].apply(lambda x: x[:4] if x is not np.nan else x), df21[f'팀원{i}'].apply(lambda x: x[5:] if x is not np.nan else x)

df21.drop(['팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df21.head()

,연번,주제명,팀 코드,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
0,A1,해변 관광환경 개선 및 만족도 향상을 위한 해변 추천 서비스 제작하기,A01,1413,신재현,1420,하승훈,NaN,NaN
1,A2,해양 정보를 이용한 플레이리스트 추천 프로그램,A02,2104,윤현서,2105,이다은,NaN,NaN
2,A3,인공지능과 오픈 API를 이용한 해양 경보 어플리케이션 제작,A03,2108,김경태,2119,최준영,NaN,NaN
3,A4,부산 수산시장의 해산물 가격 판단 서비스,A04,2213,안동준,2215,장연수,NaN,NaN
4,A5,역기압 효과를 이용한 파고 높이 계산을 통해 어떻게 풍랑주의보 알리미 제작할 수 있을까?,A05,2217,정재민,2218,조현준,NaN,NaN


In [13]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리

PDF_DIR = 'OceanICT_Data/2021_OceanICT/포스터/'
file_list = os.listdir(PDF_DIR)

split_documents21 = []
a= []

for f in file_list:
  loader = PyMuPDFLoader(PDF_DIR + f)
  try:
    document = loader.load()[0]
  except:
    continue
  result = df21.loc[df21[f'연번'] == f[:-4]]

  new_document = set_metadata(result.iloc[0], document, year = 2021)
  a.append(result.iloc[0])
  split_documents21.append(new_document)

In [14]:
len(split_documents21)

86

In [15]:
import pandas as pd

team_data = './data/2023.csv'

df = pd.read_csv(team_data)
df.head()

,팀 코드,주제명,해양 관련 분야 선택,분야,팀원1 학번,팀원1 이름,팀원2 학번,팀원2 이름,팀원3 학번,팀원3 이름,기타 과목,물리학,화학,생명과학,지구과학,유튜브 링크
0,A01,아두이노와 수위조절센서를 이용한 안전 부표 제작,해양 문화와 관광 진흥,A,1418,임성훈,1406,이서경,NaN,NaN,NaN,v,NaN,NaN,v,https://youtube.com/watch?v=8TJ7Fig9mOY&featur...
1,A02,"거리와 물때, 조수간만의 차, 해양 오염 정도를 고려한 갯벌 추천",해양 문화와 관광 진흥,A,2304,이지원,2308,권민찬,NaN,NaN,NaN,NaN,NaN,V,V,https://youtu.be/6CeiASdQX94
2,A03,ACO algorithm을 이용해 2030 엑스포를 위한 ​해상 도시 계획의 교통문...,해양 문화와 관광 진흥,A,2311,김민수,2312,김채준,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtu.be/5B3b9EZyLdU
3,A04,기후위기와 지역별 해적의 상관관계,해양 문화와 관광 진흥,A,2313,도영운,2314,신승창,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/watch?v=gAL6a0S56UM&t=3
4,A05,인공지능아~ 해운대 까페에서 먹을 음료 알려줘,해양 문화와 관광 진흥,A,2318,정민영,2319,정보권,NaN,NaN,V(관광 서비스),NaN,NaN,NaN,NaN,https://youtu.be/7M3h2ApRLB4


In [18]:
# 단계 1: 문서 로드(Load Documents)
# 단계 2: 문서 분할(Split Documents) -> 스킵 (사유: 포스터 섞이더라)

import os
import tqdm
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리
p = re.compile('\(([^)]+)') # 괄호 안에 있는 팀코드 추출

PDF_DIR = './data/pdfs/2023/'
file_list = os.listdir(PDF_DIR)
split_documents23 = []

for f in tqdm.tqdm(file_list):
  loader = PyMuPDFLoader(PDF_DIR + f)
  document = loader.load()[0]

  team_code = p.findall(f)[0]

  result = df.loc[df['팀 코드'] == team_code]
  result.columns = ['팀 코드', '주제명', '1', '2', '팀원1학번', '팀원1이름', '팀원2학번', '팀원2이름', '팀원3학번', '팀원3이름', '기타', '물리', '화학', '생물', '지구과학', 'YOUTUBE 영상 주소']

  nplst = result.to_numpy()
  

  for i in range(5):
    if nplst[1][10 + i] in ['v', 'V']:
      subjects = nplst[0][10 + i]


  try:
    new_document = set_metadata(result.iloc[0], document, '2023')
    new_document.metadata['Subjects'] = subjects
    split_documents23.append(new_document)
  except:
    continue
    #split_documents.append(new_document)

  0%|          | 0/92 [00:00<?, ?it/s]

[['B42' '옥시벤존 농도 모니터링 프로그램을 통한 해양생태계 보호' '해양 생태계 및 환경 보존' 'B' 3203 '조수현'
  '3306' '홍수민' nan nan nan nan 'V' 'V' nan 'https://youtu.be/LX37iXYXHTA']]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
len(split_documents23)

92

In [ ]:
split_documents23

[Document(metadata={'Year': '2023', 'Team code': 'B42', 'Title': '옥시벤존 농도 모니터링 프로그램을 통한 해양생태계 보호', 'Team name': '0', 'Teammate #1 name': '조수현', 'Teammate #1 number': '3203', 'Teammate #2 name': '홍수민', 'Teammate #2 number': '3306', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0000', 'Youtube link': '0'}, page_content='Busan science high school\n2023 Ocean ICT Festival\n2023 BOIF\nYoutube 영상 QR\nB \n42\nQR 코드 영역\nQR 삽입 후 \n테두리 삭제\n옥시벤존농도모니터링프로그램을\n통한해양생태계보호\n조수밤바다홍수경보\n3203 조수현 3306 홍수민\n1. 탐구 목적\n[2022 Ocean ICT]\n해수의 옥시벤존 농도를 측정해 특정 값 이상이 되면 경고를 울리고 옥시벤존 분해 물질을 투여하는 코드를 작성하여 옥시벤존의 농도를 줄이는 탐구를 진행함.\n[2023 Ocean ICT]\n하지만, 해수의 옥시벤존을 직접적으로 분해시키는 방법은 매우 일시적인 효과를 가지며, 가성비와 효율이 좋지 못하다는 단점이 있었다.\n따라서, 이번에는 접근법에 변화를 주어 바다로 가는 사람들이나 해변에서 활동하는 사람들을 대상으로 옥시벤존과 옥티노세이트가 함유된 선크림을 사용하지\n않도록 경고하거나 대체 제품을 추천하는 프로그램을 제작하고자 하였다.\n[해양 문제]\n선크림의 주성분(옥시벤존, 옥티노세이트) :\n산호의DNA 손상유도해, 어린 산호의 기형 생성& 산호 자체 껍질의성장 막아 죽게 함\n2022 Ocean ICT Youtube 영상 →\n[융합 분야]\n정보과학 + 생명과학 + 화학\n2. 알고리즘 \n•\n옥시벤존 분해 물질(해조류(미생물)

## 2024

In [19]:
df24 = pd.read_excel('./data/2024.xlsx', sheet_name= '팀별', usecols=[i + 1 for i in range(9)], skiprows=2, nrows=98)

df24 = df24.rename(columns={'섹션\n번호': '팀 코드'})
df24.drop(0, inplace=True)
df24.drop(['학년'], axis= 1,inplace=True)

df24['팀 코드'] = df24['팀 코드'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))
df24.fillna('0000', inplace=True)

for i in range(1, 4):
    try:
        df24[f'팀원{i}학번'] = df24[f'팀원{i} 학번'].astype(int).astype(str)
        df24.rename(columns={f'팀원{i} 이름': f'팀원{i}이름'},inplace=True)
        df24.drop([f'팀원{i} 학번'],inplace=True,axis=1)
    except:
        continue

df24['주제명'] = df24['팀명']
df24['과목'] = ['' for i in range(df24.shape[0])]
df24['얘 들어갔어요'] = np.zeros(df24.shape[0],dtype=int)

In [20]:
df24_1 = pd.read_excel('./data/2024_1.xlsx', sheet_name = '팀별',usecols=[i + 2 for i in range(14)], skiprows=2, nrows=98)

# #df24 = df24.rename(columns={'섹션\n번호': '팀 코드'})
df24_1.drop(0, inplace=True)
df24_1.drop(df24_1.columns[1:8],axis=1,inplace=True)
df24_1.columns = ['팀원1 학번', '물','화','생','지','기타', '주제명']

df24_1.head(10)

,팀원1 학번,물,화,생,지,기타,주제명
1,1102.0,NaN,1,NaN,NaN,NaN,난류와 ph에 의한 갯녹음화지도 만들기
2,1103.0,1,NaN,NaN,NaN,NaN,스마트 유령어구 위치 추적 시스템
3,1105.0,1,1,NaN,NaN,NaN,폐수 유출 지점 탐색
4,1110.0,1,NaN,NaN,1,NaN,이안류와 영향인자의 상관관계 분석을 통한 이안류 예측 및 대응 알고리즘
5,1113.0,NaN,NaN,1,1,NaN,멸종위기 해양생물 개체수와 서식지역 파악하기
6,1114.0,NaN,NaN,1,NaN,NaN,딥러닝 기반 멸종 위기 어류 판별 및 보호
7,1117.0,1,NaN,NaN,NaN,NaN,전자기유도를 이용한 파도 분석 장치 및 고효율 발전기 고안
8,1203.0,NaN,NaN,NaN,1,NaN,위도별 해수의 온도와 해수 온도 상승 위험 정도 알림
9,1205.0,1,NaN,NaN,1,NaN,방사능 오염수의 확산에 따른 악영향을 최소화하기 위한 방법 찾기
10,1206.0,1,NaN,NaN,NaN,NaN,모듈형 쌍동선의 자동 결합을 위한 코드 개발


In [21]:
subject_name = ['물리','화학','생물','지구과학','기타과목']

for i in range(df24_1.shape[0]):
    subjects = list(df24_1.loc[i + 1, ['물','화','생','지','기타']])
    subject = []
    for j in range(5):
        if subjects[j] == 1:
            subject.append(subject_name[j])

    teamname = str(int(df24_1.iloc[i, 0]))
    projectname = df24_1.iloc[i, 6]
    
    df24.loc[df24[f'팀원1학번'] == teamname, '주제명'] = projectname
    print(subject)
    for j in subject:
        df24.loc[df24[f'팀원1학번'] == teamname, '과목'] = df24.loc[df24[f'팀원1학번'] == teamname, '과목'] + j

df24.head(10)

['화학']
['물리']
['물리', '화학']
['물리', '지구과학']
['생물', '지구과학']
['생물']
['물리']
['지구과학']
['물리', '지구과학']
['물리']
['지구과학']
['생물', '지구과학']
['물리', '지구과학']
['물리', '생물']
['생물']
['생물']
['화학']
['생물']
['물리', '지구과학']
['생물', '지구과학', '기타과목']
['물리']
['물리', '생물']
['물리', '기타과목']
['물리', '지구과학']
['물리']
['물리']
['생물']
['생물', '지구과학']
['물리']
['물리', '지구과학']
['물리']
['물리']
['생물', '지구과학']
['물리']
['물리', '지구과학']
['생물', '기타과목']
['물리']
['생물', '기타과목']
['지구과학', '기타과목']
['물리', '기타과목']
['화학', '지구과학']
['생물']
['물리', '지구과학']
['물리', '지구과학']
['지구과학']
['생물', '지구과학']
['지구과학']
['물리', '지구과학']
['물리']
['물리', '지구과학']
['생물', '지구과학']
['생물']
['생물']
['물리', '생물', '지구과학']
['지구과학', '기타과목']
['화학', '지구과학']
['기타과목']
['생물']
['물리', '생물', '지구과학']
[]
['기타과목']
['화학', '지구과학']
['물리', '지구과학']
['지구과학', '기타과목']
['생물', '지구과학']
['물리', '지구과학']
['지구과학']
['생물', '지구과학']
['기타과목']
['생물', '지구과학']
['생물']
['생물']
['화학', '생물']
['물리']
['물리', '지구과학']
['물리', '지구과학']
['물리']
['생물', '지구과학']
['생물', '지구과학']
['생물']
['생물']
['물리', '생물']
['기타과목']
['생물']
['화학', '생물']
['화학', '생물', '기타과

,팀 코드,팀원1이름,팀원2이름,팀원3이름,팀명,팀원1학번,팀원2학번,팀원3학번,주제명,과목,얘 들어갔어요
1,B01,0000,0000,0000,클로렐라쌀밥,1102,1306,0,난류와 ph에 의한 갯녹음화지도 만들기,화학,0
2,B02,0000,0000,0000,유령인간,1103,1104,0,스마트 유령어구 위치 추적 시스템,물리,0
3,B03,0000,0000,0000,디버깅 마스터,1105,1106,0,폐수 유출 지점 탐색,물리화학,0
4,A01,0000,0000,0000,정보통신기술,1110,1112,0,이안류와 영향인자의 상관관계 분석을 통한 이안류 예측 및 대응 알고리즘,물리지구과학,0
5,B04,0000,0000,0000,별미,1111,1505,0,별미,,0
6,B05,0000,0000,0000,오진우뭐먹어나도줘,1113,1116,0,멸종위기 해양생물 개체수와 서식지역 파악하기,생물지구과학,0
7,B06,0000,0000,0000,바다의 왕자,1114,1118,0,딥러닝 기반 멸종 위기 어류 판별 및 보호,생물,0
8,D01,0000,0000,0000,바다와 고딩,1117,1409,0,전자기유도를 이용한 파도 분석 장치 및 고효율 발전기 고안,물리,0
9,B07,0000,0000,0000,SeaPerature,1203,1214,0,위도별 해수의 온도와 해수 온도 상승 위험 정도 알림,지구과학,0
10,B08,0000,0000,0000,Radioactive BSS,1205,1211,0,방사능 오염수의 확산에 따른 악영향을 최소화하기 위한 방법 찾기,물리지구과학,0


In [22]:
df24_2 = pd.read_excel('./data/2024_2.xlsx', sheet_name = '팀별',usecols=[i + 2 for i in range(14)], skiprows=3, nrows=96)

# #df24 = df24.rename(columns={'섹션\n번호': '팀 코드'})
df24_2.drop(df24_2.columns[1:3],axis=1,inplace=True)
df24_2.drop(df24_2.columns[2:],axis=1,inplace=True)

for i in range(df24_2.shape[0]):
    num = str(int(df24_2.iloc[i, 1]))
    youtubelink = df24_2.iloc[i, 0]
    df24.loc[df24[f'팀원1학번'] == num, 'YOUTUBE 영상 주소'] = youtubelink
    if df24.loc[df24[f'팀원1학번'] == num].empty:
        df24.loc[df24[f'팀원2학번'] == num, 'YOUTUBE 영상 주소'] = youtubelink

df24_2.head()

,유투브주소,Unnamed: 5
0,https://youtu.be/0Mm7suCj5Ig?si=FMAOYEXFpypYKsj8,1
1,https://youtu.be/I1YT5vt_4uQ,1
2,https://youtu.be/KNTRxJocCwA?si=ejb1yXhJDCiHGYqN,1
3,https://youtu.be/0gNUpnYtogE?si=0NyEMVpE5whb-0XX,1
4,https://www.youtube.com/watch?v=4i9Y0u8i1p0,3


In [23]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import tqdm

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 팀코드 추출
p1 = re.compile('[가-힣]+')
num = re.compile('\d{4} ?[가-힣]{2,4}')

DIR = 'OceanICT_Data/2024_OceanICT/'
file_list = os.listdir(DIR)

split_documents24 = []


for f in file_list:
    files = os.listdir(DIR + f)
    for i in files:
        if i.endswith('.pdf'):
            loader = PyMuPDFLoader(DIR + f +'/' + i)
        
        elif i.endswith(('.pptx', '.ppt')):
            loader = UnstructuredPowerPointLoader(DIR + f +'/' + i)
        
        else: continue

    number = p.findall(i)

    document = loader.load()[0]

    for j in number:
        result = df24.loc[df24[f'팀원1학번'] == j]
        print(1)
        if not result.empty:
            df24.loc[df24[f'팀원1학번'] == j, '얘 들어갔어요'] = 1
            break
    print(i)
    nnum = num.findall(i)

    
    
    try:
        print(result)
        new_document = set_metadata(result.iloc[0], document, year = 2024)
        new_document.metadata["Subjects"] = result.iloc[0, 9]
        for j in range(len(nnum)):
            new_document.metadata[f'Teammate #{j + 1} name'] = p1.search(nnum[j]).group()
            new_document.metadata[f'Teammate #{j + 1} number'] = p.search(nnum[j]).group()
        split_documents24.append(new_document)
    except:
        continue

1
파도위의혁명단_2509김민규_2511김준민(썸네일).png
   팀 코드 팀원1이름 팀원2이름 팀원3이름        팀명 팀원1학번 팀원2학번 팀원3학번                    주제명  \
76  D12  0000  0000  0000  파도위의 혁명단  2509  2511     0  머신러닝을 이용한 해상풍력 발전량 예측   

        과목  얘 들어갔어요 YOUTUBE 영상 주소  
76  물리지구과학        0           NaN  
1
부희주들_2304정유진_2305하은결.pptx
   팀 코드 팀원1이름 팀원2이름 팀원3이름    팀명 팀원1학번 팀원2학번 팀원3학번                        주제명  \
55  B30  0000  0000  0000  0000  2304  2305     0  해양 유동 역학과 생물 활동의 상호작용 모델링   

          과목  얘 들어갔어요 YOUTUBE 영상 주소  
55  지구과학기타과목        0           NaN  
1
빈포인트_2414유예준_2416임성훈.pdf
   팀 코드 팀원1이름 팀원2이름 팀원3이름        팀명 팀원1학번 팀원2학번 팀원3학번  \
69  A10  0000  0000  0000  binpoint  2414  2416     0   

                        주제명    과목  얘 들어갔어요 YOUTUBE 영상 주소  
69  무단 투기된 쓰레기 신고 어플리케이션 개발  기타과목        0           NaN  
1
1
잠수함_1405장윤_1502김소원_1503박지유.png
   팀 코드 팀원1이름 팀원2이름 팀원3이름              팀명 팀원1학번 팀원2학번 팀원3학번  \
30  C05  0000  0000  0000  blue submarine  1502  1503  1405   

                            주제명      과목  얘 들어

In [24]:
import pprint

for i in split_documents24:
    print(i.metadata[f'Subjects'])

물리지구과학
지구과학기타과목
기타과목
물리지구과학
물리
생물
물리생물지구과학
화학생물
화학
생물생물
생물
생물
물리
생물
생물
지구과학기타과목
화학
화학
생물지구과학
지구과학
생물기타과목
생물기타과목
물리
생물지구과학
물리
화학지구과학
물리지구과학
물리화학
생물
물리지구과학
생물지구과학
물리지구과학
생물지구과학
생물
생물
물리기타과목
지구과학기타과목

물리
지구과학
생물
물리
물리
생물
생물
기타과목
생물지구과학기타과목
물리생물지구과학
생물지구과학
물리지구과학

생물
물리
생물
지구과학
물리지구과학
물리지구과학
생물
화학
기타과목
기타과목
생물기타과목
물리화학
물리지구과학
화학지구과학
물리생물
물리
생물지구과학
물리
생물지구과학
물리화학
지구과학
물리생물
물리
물리지구과학
생물지구과학
물리
생물
물리
물리지구과학


In [25]:
split_documents24

[Document(metadata={'Year': '2024', 'Team code': 'D12', 'Title': '머신러닝을 이용한 해상풍력 발전량 예측', 'Team name': '파도위의 혁명단', 'Teammate #1 name': '김민규', 'Teammate #1 number': '2509', 'Teammate #2 name': '김준민', 'Teammate #2 number': '2511', 'Teammate #3 name': '0000', 'Teammate #3 number': '0', 'Youtube link': 'nan', 'Subjects': '물리지구과학'}, page_content='Busan science high school\n\n2024 Ocean ICT Festival\n\n2023 BOIF\n\nD\n\n12\n\nYoutube 영상 QR\n\n해상풍력 적합 지역 찾기&머신러닝을 이용한 발전량 예측\n\n파도위의혁명단(2509김민규, 2511김준민)\n\n기상학적 요인들을 고려하여 해상 풍력 발전량을 예측하여 많은 이점을 얻을 수 있을 것 이라고 생각하여 이 주제를 정했다. 본 탐구는 해상 풍력 발전으로 전력 공급 안정성을 확보하고,\n\n 또 그를 통해 자원 관리의 효율성을 확보하도록 할 수 있다.(스마트 그리드)\n\n주제 선정 동기\n\n해상풍력에 적합한 지역을 찾을 때, 실제 평균 풍속과 가장 비슷한 확률 분포 모델을 사용 해야한다. 이때 확률 질량 함수를 계산해야 하므로 수학에서 통계 파트가 적용된다. 또 유체 지구과학의 \n\n바람장 파트와도 밀접한 연관이 있다고 할 수 있다. 또, 사영한 풍속값을 사용할 것이기 때문에, \n\n물리에서 벡터, 또 공간도형에서 나오는 정사영 파트 까지 수학과의 연관성을 연장할 수 있다.\xa0\n\n융합분야\n\n개요\n\n서귀포, 신안, 추자도, 마라도, 칠발도의  2024풍속데이터를 받는다.\n\n해상풍력발전기의 방향을 변수로 하여 풍속분포를 GUMbell분포(혼합분포)로 근사한

In [26]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [27]:
import pprint 

for i in split_documents19:
    print(i.metadata)

{'Year': '2019', 'Team code': 'B01', 'Title': '유해성 적조 피해, 줄일 수 없을까?', 'Team name': '0', 'Teammate #1 name': ' 김민선', 'Teammate #1 number': '2101', 'Teammate #2 name': ' 조정민', 'Teammate #2 number': '2304', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0000', 'Youtube link': '0'}
{'Year': '2019', 'Team code': 'C01', 'Title': '대형해조류 제거와 증식 억제, 두 마리 토끼를 다 잡을 수 없을까?', 'Team name': '0', 'Teammate #1 name': ' 양유진', 'Teammate #1 number': '2105', 'Teammate #2 name': ' 양서영', 'Teammate #2 number': '2505', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0000', 'Youtube link': '0'}
{'Year': '2019', 'Team code': 'B02', 'Title': '빈산수소괴로 인한 생태계 파괴를 해결해 볼까요?', 'Team name': '0', 'Teammate #1 name': ' 민황선', 'Teammate #1 number': '2301', 'Teammate #2 name': ' 임지혜', 'Teammate #2 number': '2106', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0000', 'Youtube link': '0'}
{'Year': '2019', 'Team code': 'B03', 'Title': '해양쓰레기 검출에 특화된 이미지 학습 알고리즘을 만들 수 있을까?', 'Team name': '0', 'Teammate #1 name': ' 강신재',

In [28]:
split_documents24

[Document(metadata={'Year': '2024', 'Team code': 'D12', 'Title': '머신러닝을 이용한 해상풍력 발전량 예측', 'Team name': '파도위의 혁명단', 'Teammate #1 name': '김민규', 'Teammate #1 number': '2509', 'Teammate #2 name': '김준민', 'Teammate #2 number': '2511', 'Teammate #3 name': '0000', 'Teammate #3 number': '0', 'Youtube link': 'nan', 'Subjects': '물리지구과학'}, page_content='Busan science high school\n\n2024 Ocean ICT Festival\n\n2023 BOIF\n\nD\n\n12\n\nYoutube 영상 QR\n\n해상풍력 적합 지역 찾기&머신러닝을 이용한 발전량 예측\n\n파도위의혁명단(2509김민규, 2511김준민)\n\n기상학적 요인들을 고려하여 해상 풍력 발전량을 예측하여 많은 이점을 얻을 수 있을 것 이라고 생각하여 이 주제를 정했다. 본 탐구는 해상 풍력 발전으로 전력 공급 안정성을 확보하고,\n\n 또 그를 통해 자원 관리의 효율성을 확보하도록 할 수 있다.(스마트 그리드)\n\n주제 선정 동기\n\n해상풍력에 적합한 지역을 찾을 때, 실제 평균 풍속과 가장 비슷한 확률 분포 모델을 사용 해야한다. 이때 확률 질량 함수를 계산해야 하므로 수학에서 통계 파트가 적용된다. 또 유체 지구과학의 \n\n바람장 파트와도 밀접한 연관이 있다고 할 수 있다. 또, 사영한 풍속값을 사용할 것이기 때문에, \n\n물리에서 벡터, 또 공간도형에서 나오는 정사영 파트 까지 수학과의 연관성을 연장할 수 있다.\xa0\n\n융합분야\n\n개요\n\n서귀포, 신안, 추자도, 마라도, 칠발도의  2024풍속데이터를 받는다.\n\n해상풍력발전기의 방향을 변수로 하여 풍속분포를 GUMbell분포(혼합분포)로 근사한

In [29]:
vectorstore = Chroma.from_documents(
        documents=split_documents24,
        embedding=OpenAIEmbeddings(),
        persist_directory=f'db/chroma_2024_pdfs'
    )